# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import graphlab
import math
import string

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [5]:
products = graphlab.SFrame('output11_v22.csv')

Unable to parse line "9392,"RT @RunyonHailey: What are we gonna throw @ football games? No baby powder or corn starch. "

Unable to parse line "19239,"RT @Zedd: so now u know:"

Unable to parse line "- i'm in ibiza"

Unable to parse line "FLOUR! "

Unable to parse line "- I'm in my underwear"

...

Unable to parse line "- ^ deadline is today"

Unable to parse line "52,"*blinks in shock before sagging a bit*"

8355 lines failed to parse correctly

Finished parsing file C:\Users\Anton\output11_v22.csv

Parsing completed. Parsed 16735 lines in 0.102092 secs.

Now, let us see a preview of what the dataset looks like.

In [6]:
products

X1,text,time,prct_change,sentiment
0,RT @melissasraja:Valentines Day candy ...,Fri Sep 09 06:07:27 +00002016 ...,0,0
4,#corn sex milton twinsporn ...,Fri Sep 09 06:07:51 +00002016 ...,0,0
8,The movies. Thetemperature. The hay ...,Fri Sep 09 06:07:56 +00002016 ...,0,0
9,RT @ESPNNBA: Corn Mazes:NBA Championship edit ...,Fri Sep 09 06:07:58 +00002016 ...,0,0
11,Corn on the Cob - YouSuck at Cooking (episode ...,Fri Sep 09 06:08:03 +00002016 ...,0,0
15,"Macaroni soup withpotatoes, carrots and ...",Fri Sep 09 06:08:28 +00002016 ...,0,0
16,Mainly because I REALLYwanna give myself some ...,Fri Sep 09 06:08:32 +00002016 ...,0,0
17,The corn Spainhttps://t.co/4gigizQOtZ ...,Fri Sep 09 06:08:45 +00002016 ...,0,0
19,RT @YukiJorouge: [asmr]weeb sucks a corn dog ...,Fri Sep 09 06:08:51 +00002016 ...,0,0
20,@aatikah98 ya! Pop cornhahaha ...,Fri Sep 09 06:09:02 +00002016 ...,0,0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [7]:
products[269]

{'X1': '551',
 'prct_change': 0L,
 'sentiment': 0L,
 'text': "RT @sidkneecap: y do ppl keep telling me it looks like I've lost weight like bitch WHERE I look like a corn dog",
 'time': 'Fri Sep 09 06:46:41 +0000 2016'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [9]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_puctuation = products['text'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_puctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [10]:
products[269]['word_count']

{'a': 1L,
 'bitch': 1L,
 'corn': 1L,
 'do': 1L,
 'dog': 1L,
 'i': 1L,
 'it': 1L,
 'ive': 1L,
 'keep': 1L,
 'like': 3L,
 'look': 1L,
 'looks': 1L,
 'lost': 1L,
 'me': 1L,
 'ppl': 1L,
 'rt': 1L,
 'sidkneecap': 1L,
 'telling': 1L,
 'weight': 1L,
 'where': 1L,
 'y': 1L}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [11]:
products = products[products['sentiment'] != 0]
len(products)

15218

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [12]:
#products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
#products

Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [13]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

12175
3043


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [14]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

Logistic regression:

--------------------------------------------------------

Number of examples          : 12175

Number of classes           : 2

...

| 5         | 8        | 1.000000  | 1.297403     | 0.949979          |

| 6         | 9        | 1.000000  | 1.331433     | 0.948501          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [15]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 23210
Number of examples             : 12175
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 23209

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 10
Solver status                  : TERMINATED: Iteration limit reached.
Training time (sec)            : 1.5036

Settings
--------
Log-likelihood                 : 1154.8262

Highest Positive Coefficients
-----------------------------
word_count[httpstconr3lvp32y7] : 9.4529
word_count[mitchelbosko1]      : 7.9025
word_count[httpstco4mzcwgrgqi] : 7.3323
word_count[atidavefogel]       : 7.1902
word_count[anarojas]           : 7.1864

Lowest Negative Coefficients
----------------------------
word_count[httpstcogctblwskbc] : -21

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [16]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [17]:
num_positive_weights = len(weights[weights['value'] >= 0])
num_negative_weights = len(weights[weights['value'] < 0])

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 17525 
Number of negative weights: 5685 


**Quiz question:** How many weights are >= 0?  68419 

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [19]:
sample_test_data = test_data[10:13]
print sample_test_data['sentiment']
sample_test_data

[-1L, -1L, -1L]


X1,text,time,prct_change,sentiment,word_count
917,RT @busquetsmarcelo:$CORN #corn trading ...,Fri Sep 09 07:12:00 +00002016 ...,0,-1,"{'alza': 1L, 'rt': 1L,'ahi': 1L, 'para': 1L, ..."
941,"Corn dogs, look like thedogs are wearing .. down ...",Fri Sep 09 07:14:38 +00002016 ...,0,-1,"{'wearing': 1L, 'look':1L, 'corn': 1L, ..."
971,RT @AgsPlayers: ExpectedChinese corn crop dec ...,Fri Sep 09 07:16:27 +00002016 ...,0,-1,"{'rt': 1L, 'decline': 1L,'usda': 1L, 'chinese': ..."


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [20]:
sample_test_data[0]['text']

"RT @busquetsmarcelo: $CORN  #corn   trading 30'   veremos si retrae a 324 para ahi si comenzar a perfilar alza consistente. https://t.co/fS\x85"

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [27]:
sample_test_data[2]['text']

'RT @AgsPlayers: Expected Chinese corn crop decline less than USDA ideas - RJO'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [22]:
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print scores

[3.3684470938060156, -2.7209859174219257, -2.9440030809974265]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [23]:
import numpy as np
ytag = np.sign(scores)
print ytag

[ 1. -1. -1.]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [24]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data)

Class predictions according to GraphLab Create:
[1L, -1L, -1L]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [25]:
import math
probytag = math.e**(-scores)
probytagscores = 1/(1+probytag)
print probytagscores

[0.9667037430216904, 0.06174632371879778, 0.050020709224901226]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [26]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

Class predictions according to GraphLab Create:
[0.9667037430216904, 0.06174632371879779, 0.0500207092249012]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review? Third

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [37]:
#print sentiment_model.predict(test_data, output_type='probability')
test_data['probability_prediction']=sentiment_model.predict(test_data, output_type='probability')
top20 = test_data.topk('probability_prediction', k=20)
#print top20.column_names()
print top20[10:20]

+-------+-------------------------------+--------------------------------+-------------+
|   X1  |              text             |              time              | prct_change |
+-------+-------------------------------+--------------------------------+-------------+
| 26516 | RT @congress_nagpur: Ex Un... | Sat Sep 10 03:39:33 +0000 2016 |      0      |
| 11779 | RT @congress_nagpur: Ex Un... | Fri Sep 09 17:00:18 +0000 2016 |      0      |
|  9384 | #d?deblomster 3) s?t Allan... | Fri Sep 09 15:33:00 +0000 2016 |      0      |
| 11924 | RT congress_nagpur: Ex Uni... | Fri Sep 09 17:05:31 +0000 2016 |      0      |
| 26749 | @SampleAgri @DallasSansom ... | Sat Sep 10 03:51:56 +0000 2016 |      0      |
| 13768 | Idgaf if I start an argume... | Fri Sep 09 18:23:27 +0000 2016 |      0      |
| 12576 | RT @mdgblogger: Vegan Corn... | Fri Sep 09 17:30:01 +0000 2016 |      0      |
| 27086 | @overtflow overtflow caugh... | Sat Sep 10 04:12:23 +0000 2016 |      0      |
| 20283 | Dusk- the g

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [29]:
bottom20 = test_data.topk('probability_prediction', k=20, reverse = True)
print bottom20[10:20]

+-------+-------------------------------+--------------------------------+-------------+
|   X1  |              text             |              time              | prct_change |
+-------+-------------------------------+--------------------------------+-------------+
|  1068 | RT @seemaadhikari: ?????? ... | Fri Sep 09 07:24:09 +0000 2016 |      0      |
|  3781 | RT @MrsJellySantos: Twizzl... | Fri Sep 09 11:45:39 +0000 2016 |      0      |
| 13312 | RT @UWExtensionANRE: Check... | Fri Sep 09 18:01:38 +0000 2016 |      0      |
|  5498 | RT @RootedinVermont: Cabot... | Fri Sep 09 13:09:04 +0000 2016 |      0      |
|  1411 | Creppy fog over the corn f... | Fri Sep 09 07:56:11 +0000 2016 |      0      |
|  3251 | RT @WISH_TV: Beasley's Cor... | Fri Sep 09 11:08:55 +0000 2016 |      0      |
|  3242 | RT @WISH_TV: Beasley's Cor... | Fri Sep 09 11:08:17 +0000 2016 |      0      |
|  5227 | RT @PHLBizJournal: 'Fancy'... | Fri Sep 09 12:55:36 +0000 2016 |      0      |
|  6949 | Traders pre

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [30]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    correct_examples = []
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    for i in range(0,len(predictions)):
        if predictions[i] != true_labels[i]: continue
        else:
            correct_examples.append(predictions[i])
    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = len(correct_examples)/len(predictions)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [31]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.8149852119618797

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76). 1.00

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better? Yes

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [50]:
significant_words = ['bullish', 'bearish', 'rally']

In [51]:
len(significant_words)

3

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [52]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [53]:
train_data[0]['text']

'A superhero corn cob with the sensibility to beat up spades in order to defeat birds.'

The **word_count** column had been working with before looks like the following:

In [54]:
print train_data[0]['word_count']

{'a': 1L, 'spades': 1L, 'to': 2L, 'beat': 1L, 'corn': 1L, 'defeat': 1L, 'up': 1L, 'birds': 1L, 'sensibility': 1L, 'cob': 1L, 'in': 1L, 'superhero': 1L, 'the': 1L, 'with': 1L, 'order': 1L}


Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [55]:
print train_data[0]['word_count_subset']

{}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [56]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 12175

Number of classes           : 2

Number of feature columns   : 1

...

| 3         | 4        | 0.043039     | 0.804682          |

| 4         | 5        | 0.056051     | 0.804682          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 3
Number of examples             : 12175
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 2

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 4
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.0601

Settings
--------
Log-likelihood                 : 6012.1515

Highest Positive Coefficients
-----------------------------
(intercept)                    : 1.4156
word_count_subset[rally]       : 0.6638

Lowest Negative Coefficients
----------------------------
word_count_subset[bearish]     : -2.1088

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [57]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.7962536970095301

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [58]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.41563657229,0.0228703252504
word_count_subset,rally,1,0.66379109091,1.06090098887
word_count_subset,bearish,1,-2.10876816318,1.22495520614


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [59]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)
#sentiment_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+---------+-------+----------------+-----------------+
|        name       |  index  | class |     value      |      stderr     |
+-------------------+---------+-------+----------------+-----------------+
|    (intercept)    |   None  |   1   | 1.41563657229  | 0.0228703252504 |
| word_count_subset |  rally  |   1   | 0.66379109091  |  1.06090098887  |
| word_count_subset | bearish |   1   | -2.10876816318 |  1.22495520614  |
+-------------------+---------+-------+----------------+-----------------+
[3 rows x 5 columns]



**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [60]:
test_data['probability_prediction']=simple_model.predict(test_data, output_type='probability')
top20 = test_data.topk('probability_prediction', k=20)
#print top20.column_names()
print top20[10:20]

+-----+-------------------------------+--------------------------------+-------------+
|  X1 |              text             |              time              | prct_change |
+-----+-------------------------------+--------------------------------+-------------+
| 802 |       Candy Corn Vampire      | Fri Sep 09 07:01:07 +0000 2016 |      0      |
| 941 | Corn dogs, look like the d... | Fri Sep 09 07:14:38 +0000 2016 |      0      |
| 917 | RT @busquetsmarcelo: $CORN... | Fri Sep 09 07:12:00 +0000 2016 |      0      |
| 912 | King Corn - Aaron Woolf | ... | Fri Sep 09 07:11:38 +0000 2016 |      0      |
| 883 | Pops Arcadia Corn Maze htt... | Fri Sep 09 07:08:33 +0000 2016 |      0      |
| 874 | Reasons I like mexican Cok... | Fri Sep 09 07:07:23 +0000 2016 |      0      |
| 870 | Slob On My Knob Like Corn ... | Fri Sep 09 07:06:59 +0000 2016 |      0      |
| 845 | You can do it... grow and ... | Fri Sep 09 07:04:47 +0000 2016 |      0      |
| 822 | RT @SweetDealsUK: OPEN 9 -... | Fri

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [64]:
test_data2 = test_data[test_data['word_count_subset'] != {}]
test_data2['text']

dtype: str
Rows: ?
['AM markets: grains rally loses vim as weekend and Wasde loom https://t.co/qdxU0UzYil Corn, wheat', 'AM markets: grains rally loses vim as weekend and Wasde loom https://t.co/lHKo5u81pk Corn, wheat', '"I don�t feel a small reduction is bullish, but it is supportive at the least." #corn #yield https://t.co/nFIMCiT1tF', '#corn rally off 3.16 level our analysis isolated. Settled &gt; weekly trend; 3.302 for week of 09/12. use as pivot. https://t.co/7z6b57T84A', 'ITALY NARDI RALLY DEEP CORN 330 MM STEERING WHEEL MAHOGANY WOOD POLISH SPOKE https://t.co/yrylbX7SbW https://t.co/0ZbnHWuJ9U', ... ]

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [87]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [88]:
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set? Sentiment model

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [89]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [90]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set? Simple Model

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [76]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [102]:
num_positive_test  = (test_data['sentiment'] == +1).sum()
num_negative_test = (test_data['sentiment'] == -1).sum()
print num_positive_test/len(test_data)

0.842782577394


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?

In [98]:
Yes

NameError: name 'Yes' is not defined